Training, Validation and testing dataset were labeled by batch manually

In [19]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Diffusion_denoising_thin_slice.functions_collection as ff

In [ ]:
from calendar import c
from math import ceil


data = pd.read_excel('/host/d/Data/scores_ZhennongChen.xlsx')
print(data.shape)
column_formula = data['Formula']
# round up
data['Formula_new'] = column_formula.apply(lambda x: ceil(x))

column_speedup = data['Speedup']
# round up
data['Speedup_new'] = column_speedup.apply(lambda x: ceil(x))

column_efficiency = data['Efficiency']
# round up
data['Efficiency_new'] = column_efficiency.apply(lambda x: ceil(x))

column_line = data['Line']
# round up
data['Line_new'] = column_line.apply(lambda x: ceil(x))
data.to_excel('/host/d/Data/scores_ZhennongChen_rounded.xlsx', index=False)

(425, 12)


ValueError: cannot convert float NaN to integer

### read patient list

In [10]:
patient_list = pd.read_csv('/host/d/Data/PCCT/Patient_lists/raw_info.csv',dtype={'Patient_ID':str})
patient_list = patient_list[patient_list['NumChannels'] == 1]
print(f"Number of single channel patients: {len(patient_list)}")


Number of single channel patients: 33


In [13]:
batch_column =[]
for i in range(len(patient_list)):
    if i<22:
        batch_column.append(0)
    elif i>=22 and i<25:
        batch_column.append(1)
    else:
        batch_column.append(2)
patient_list['batch'] = batch_column
patient_list.to_excel('/host/d/Data/PCCT/Patient_lists/PCCT_batched.xlsx', index=False)

# build list for simulations

In [18]:
patient_sheet = pd.read_excel(os.path.join('/host/d/Data/PCCT/Patient_lists/PCCT_batched.xlsx'),dtype={'Patient_ID': str})

data_path = '/host/d/Data/PCCT/data/soft_thins'

results = []
for i in range(0, len(patient_sheet)):
    patient_id = patient_sheet['Patient_ID'][i]
    patient_subid = patient_id
    # patient_subid = patient_sheet['Patient_subID'][i]
    batch = patient_sheet['batch'][i]
 
    
    simulation_file = os.path.join(data_path, patient_id,'soft_thins_0_noblank.nii.gz')
    assert os.path.exists(simulation_file), f"Simulation file does not exist: {simulation_file}"
    simulation_img = nb.load(simulation_file).get_fdata()
    x_shape, y_shape, slice_num = simulation_img.shape
    # ground_truth_file = simulation_file
            

    results.append([batch,patient_id, patient_subid,simulation_file, x_shape, y_shape, slice_num])

    df = pd.DataFrame(results, columns=['batch','Patient_ID','Patient_subID', 'noise_file', 'x_shape', 'y_shape', 'slice_num'])
    df.to_excel(os.path.join('/host/d/Data/PCCT/Patient_lists','PCCT_split.xlsx'), index=False)





# build list for distilled model

In [3]:
patient_sheet = pd.read_excel(os.path.join('/host/d/Data/brain_CT/Patient_lists/fixedCT_static_shuffled_batched.xlsx'),dtype={'Patient_ID': str, 'Patient_subID': str})
simulation_num = 1

link = 'xjtlu' # 'local' or 'NAS'
if link == 'local':
    data_path1 = '/workspace/Documents/Data/denoising'
elif link == 'NAS':
    data_path1 = '/mnt/camca_NAS/denoising/Data'
elif link == 'xjtlu':
    data_path1 = '/host/d/projects/denoising/models/unsupervised_gaussian_brainCT'

results = []
for i in range(0, len(patient_sheet)):
    patient_id = patient_sheet['Patient_ID'][i]
    patient_subid = patient_sheet['Patient_subID'][i]
    batch = patient_sheet['batch'][i]
    print(f"Processing patient {patient_id} {patient_subid}...")


    for n in range(0,simulation_num):
        simulation_file = os.path.join(data_path1,'pred_images/',patient_id, patient_subid, 'random_'+str(n), 'epoch61_1/condition_img.nii.gz')

        ground_truth_file = os.path.join(data_path1,'pred_images/',patient_id,patient_subid,'random_'+str(n), 'epoch61avg/pred_img_scans20.nii.gz')

        results.append([batch,patient_id, patient_subid, n,simulation_file, ground_truth_file])#, simulation_img.shape])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'Patient_subID', 'random_num', 'noise_file', 'ground_truth_file'])#, 'image_shape'])
        df.to_excel(os.path.join('/host/d/Data/brain_CT/Patient_lists/fixedCT_static_distilled_model_train_test_' + link +'.xlsx'), index=False)

Processing patient 00139437 0000258390...
Processing patient 00019599 0000029506...
Processing patient 00148611 0000455363...
Processing patient 00173704 0000455358...
Processing patient 00030597 0000455473...
Processing patient 00214862 0000455530...
Processing patient 00004038 0000455420...
Processing patient 00214775 0000455224...
Processing patient 00214865 0000455504...
Processing patient 00214905 0000455573...
Processing patient 00214815 0000455367...
Processing patient 00214925 0000455633...
Processing patient 00214918 0000455613...
Processing patient 00214919 0000455609...
Processing patient 00214864 0000455532...
Processing patient 00010431 0000019300...
Processing patient 00214935 0000455654...
Processing patient 00010536 0000455747...
Processing patient 00058992 0000455611...
Processing patient 00010795 0000455825...
Processing patient 00171178 0000455242...
Processing patient 00093479 0000455522...
Processing patient 00008527 0000455795...
Processing patient 00154227 000045